# Creatig a object with the yolo model transformed into a MobileNet SSD

In [165]:
import cv2
import cvzone
import math
from itertools import combinations

import numpy as np
import os
import matplotlib.pyplot as plt
from cvzone.FaceMeshModule import FaceMeshDetector
import imutils

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
prototxtPath_face = os.path.sep.join(["face_detector", "deploy.prototxt"])
weightsPath_face = os.path.sep.join(["face_detector",
                                "res10_300x300_ssd_iter_140000.caffemodel"])

print("[INFO] loading person detector model...")
# Paths for MobileNet trained on caffe for person detection
prototxtPath = os.path.sep.join(["person_detector", "SSD_MobileNet_prototxt.txt"])
weightsPath = os.path.sep.join(["person_detector",
                                "SSD_MobileNet.caffemodel"])

def euclidean_dist(p1, p2):
    """
    p1, p2 = two points for calculating Euclidean Distance

    :return:
    dst = Euclidean Distance between two 2d points
    """
    if type(p1) is int or type(p1) is int:
        dst = math.sqrt(p1 ** 2 + p2 ** 2)
    else:
        dst = math.sqrt((p2[0] - p1[0] ) ** 2 + (p2[1] - p1[1])** 2)
    
    return dst

class Detector():
    
    def __init__(self):
        # Initialize our Face LandMark detector
        self.faceLandMarkDetector = FaceMeshDetector()
        # Initialize model for person detection
        self.net_person_detector = cv2.dnn.readNet(prototxtPath, weightsPath)
        # Initialize model for face detection
        self.net_face_detector = cv2.dnn.readNet(prototxtPath_face, weightsPath_face)
        # Initialize model for detect face mask
        self.facemask_model = load_model("mask_detector.model")
        
        
        # Detected Face Land Marks
        self.detected_facelandmarks = list()
        self.red_zone_list = []  # List containing which Object id is in under threshold distance condition.
        self.red_line_list = []
        self.centroid_dict = dict()
        
        # Distances from faces to camera
        self.distances = list()
        
        # Detectec Faces
        self.detected_faces = list()
        
    def detect_distance_from_camera(self, img):
        #detect the faceMesh
        img, self.detected_facelandmarks = self.faceLandMarkDetector.findFaceMesh(img, draw=False)
    
        self.distances = list()
    
        if len(self.detected_facelandmarks) > 0:
            for face in self.detected_facelandmarks:
                pointLeft = face[145]
                pointRight = face[374]

                w = self.faceLandMarkDetector.findDistance(pointLeft, pointRight)[0]
                # Trying to find the deep of the focal lent
                W = 6.3 #avergae distance between the eyes
                #d = 30
                #f = (w*d)/W
                f = 540
                #cv2.putText(img, str(f), (pointLeft),cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0,0), 2)

                # Finding the distance from the focal point
                d = (W*f)/w

                # Create a list with the coordinates of the eyes and the distance
                self.distances.append([d, pointLeft, pointRight])
        return self.distances
    
    def detect_sd(self, frame, distances):
        frame = imutils.resize(frame, width=400)
        (h, w) = frame.shape[:2]

        #Converting Frame to Blob
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 
            0.007843, (300, 300), 127.5)

        #Passing Blob through network to detect and predict
        self.net_person_detector.setInput(blob)
        detections = self.net_person_detector.forward()
        detections = np.array([detection for detection in detections[0][0] if sum(detection) !=0])

        if len(detections) > 0:  # At least 1 detection in the image and check detection presence in a frame
            self.centroid_dict = dict()  # Function creates a dictionary and calls it centroid_dict
            distances_dict = dict()
            objectId = 0  # We inialize a variable called ObjectId and set it to 0

            for detection in detections:  # In this if statement, we filter all the detections for persons only
                # Check for the only person name tag 
                #name_tag = detection['name']  # Coco file has string of all the names
                #if name_tag == 'person':
                if detection[1] == 15:

                    #Extracting the confidence of predictions
                    confidence = detection[2]

                    #Filtering out weak predictions
                    if confidence > 0.7:

                        #Computing the (x,y) - coordinates of the bounding box  

                        #Extracting bounding box coordinates
                        box = detection[3:7] * np.array([w, h, w, h])
                        (xmin, ymin, xmax, ymax) = box.astype("int")

                        #Drawing the prediction and bounding box
                        label = "{}: {:.2f}%".format('person', confidence * 100)

                        y = ymin - 15 if ymin - 15 > 15 else ymin + 15
                        
                        #Store the center points of the detections
                        x = int(xmin + (xmax - xmin) / 2)
                        y = int(ymin + (ymax - ymin) / 2)
                        self.centroid_dict[objectId] = [x, y, int(xmin), int(ymin), int(xmax),
                                                   int(ymax)]  # Create dictionary of tuple with 'objectId' as the index center points and bbox
                        #[for distance in distances]
                        distances_dict[objectId] = [distance for distance in distances if xmin < distance[1][0] < xmax and ymin < distance[1][0] < ymax]
                        objectId += 1  # Increment the index for each detection


            # Determine which person are close to each other
            print('Eye distances: ', distances_dict)
            print('Persons recogized: ', self.centroid_dict)
            self.red_zone_list = []  # List containing which Object id is in under threshold distance condition.
            self.red_line_list = []
            for (id1, p1), (id2, p2) in combinations(self.centroid_dict.items(),2):  # Get all the combinations of close detections, #List of multiple items - id1 1, points 2, 1,3
                dx, dy = p1[0] - p2[0], p1[1] - p2[1]  # Check the difference between centroid x: 0, y :1
                
                #socialdist_px = euclidean_dist(p1[0:2], p2[0:2])  # Calculates the Euclidean distance

                socialdist_px = euclidean_dist(dx, dy)
                socialdist_estimated = 0
                
                distance_bepx1 = 0
                distance_bepx2 = 0
                no_eye_ref = False

                distance_fc1 = 0
                distance_fc2 = 0

                if id1 in distances_dict:
                    if len(distances_dict[id1]) > 0:
                        distance_bepx1 = euclidean_dist(distances_dict[id1][0][1], distances_dict[id1][0][2])
                        distance_fc1 = distances_dict[id1][0][0]
                if id2 in distances_dict:
                    if len(distances_dict[id2]) > 0:
                        distance_bepx2 = euclidean_dist(distances_dict[id2][0][1], distances_dict[id2][0][2])
                        distance_fc1 = distances_dict[id2][0][0]

                if distance_bepx1 > distance_bepx2:
                    socialdist = socialdist_px*6.3/distance_bepx1
                    socialdist_estimated = math.sqrt((distance_fc1-distance_fc2)**2 + (socialdist)**2)
                elif distance_bepx1 < distance_bepx2:
                    socialdist = socialdist_px*6.3/distance_bepx2
                    socialdist_estimated = math.sqrt((distance_fc2-distance_fc1)**2 + (socialdist)**2)
                else:
                    socialdist_estimated = socialdist_px
                    no_eye_ref = True

                self.centroid_dict[id1].append(socialdist_estimated)
                self.centroid_dict[id1].append(socialdist_px)
                
                self.centroid_dict[id2].append(socialdist_estimated)
                self.centroid_dict[id2].append(socialdist_px)
                
                ### Referencing horizontal distances to between eyes distance (6.3cm)
                if (socialdist_estimated < 150 and not no_eye_ref) or (socialdist_estimated < 75 and no_eye_ref):  # Set our social distance threshold - If they meet this condition then..
                    if id1 not in self.red_zone_list:
                        self.red_zone_list.append(id1)  # Add Id to a list
                        self.red_line_list.append(p1[0:2])  # Add points to the list
                        #red_zone_dists.append(socialdist)
                    if id2 not in self.red_zone_list:
                        self.red_zone_list.append(id2)  # Same for the second id
                        self.red_line_list.append(p2[0:2])
        return frame

    def detect_face_mask(self, img):
        # construct a blob from the image
        blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300),
                                     (104.0, 177.0, 123.0))

        # pass the blob through the network and obtain the face detections
        #print("[INFO] computing face detections...")
        self.net_face_detector.setInput(blob)
        self.detected_faces = self.net_face_detector.forward()
        return self.detected_faces
    
    def draw_detections(self, img):
        img_copy = img.copy()

        # Draw detected eyes
        for distance in self.distances:
            color = (0, 200, 0)
            
            cv2.line(img, distance[1], distance[2], color, 3)
            cv2.circle(img, distance[1], 5, (255,0,255), cv2.FILLED)
            cv2.circle(img, distance[2], 5, (255,0,255), cv2.FILLED)
            cv2.putText(img, "{:.2f} cm".format(distance[0]), distance[1] ,cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
        
        img = imutils.resize(img, width=400)
        
        # Draw distance boxes and lines 
        for idx, box in self.centroid_dict.items():  # dict (1(key):red(value), 2 blue)  idx - key  box - value
            if idx in self.red_zone_list:  # if id is in red zone list
                #cv2.putText(frame, "{:.2f} cm".format(socialdist),(box[2], box[3]),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
                cv2.rectangle(img, (box[2], box[3]), (box[4], box[5]), (0, 0, 255),
                              2)  # Create Red bounding boxes  #starting point, ending point size of 2
            else:
                cv2.rectangle(img, (box[2], box[3]), (box[4], box[5]), (0, 255, 0),
                              2)  # Create Green bounding boxe bounding boxes  #starting
            if len(self.centroid_dict) > 1:
                cv2.putText(img, "Distance from person (estimated in cm): {:.2f} cm".format(box[6]), (box[2], box[3]), cv2.FONT_HERSHEY_SIMPLEX, 0.25, (0, 0, 0), 1, cv2.LINE_AA)  # Display Text
                cv2.putText(img, "Distance from person (pixels): {:.2f} cm".format(box[7]), (box[2], box[3]-8), cv2.FONT_HERSHEY_SIMPLEX, 0.25, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(img, "Person {}".format(idx), (box[2], box[3]-16), cv2.FONT_HERSHEY_SIMPLEX, 0.25, (0, 0, 0), 1, cv2.LINE_AA)

        for i in range(0,len(self.red_line_list)-1):
            if i%2 == 0:
                cv2.line(img, self.red_line_list[i],self.red_line_list[i+1], (0, 0, 255), 1)
                #cv2.putText(frame, "{:.2f} cm".format(socialdist),(),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
                
        # Draw face mask detected
        masks = 0
        (h, w) = img.shape[:2]
        # loop over the detections
        for i in range(0, self.detected_faces.shape[2]):
            # extract the confidence (i.e., probability) associated with
            # the detection
            confidence = self.detected_faces[0, 0, i, 2]

            # filter out weak detections by ensuring the confidence is
            # greater than the minimum confidence
            if confidence > 0.5:
                # compute the (x, y)-coordinates of the bounding box for
                # the object
                box = self.detected_faces[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # ensure the bounding boxes fall within the dimensions of
                # the frame
                (startX, startY) = (max(0, startX), max(0, startY))
                (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

                # extract the face ROI, convert it from BGR to RGB channel
                # ordering, resize it to 224x224, and preprocess it
                face = img_copy[startY:endY, startX:endX]
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face = cv2.resize(face, (224, 224))
                face = img_to_array(face)
                face = preprocess_input(face)
                face = np.expand_dims(face, axis=0)

                # pass the face through the model to determine if the face
                # has a mask or not
                (mask, withoutMask) = self.facemask_model.predict(face)[0]

                # determine the class label and color we'll use to draw
                # the bounding box and text
                label = "Mask" if mask > withoutMask else "No Mask"
                masks = masks + 0 if mask > withoutMask else masks + 1
                color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

                # include the probability in the label
                label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

                # display the label and bounding box rectangle on the output
                # frame
                cv2.putText(img, label, (startX, startY - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
                cv2.rectangle(img, (startX, startY), (endX, endY), color, 2)

        # Show how many people there are on risk
        text = "People at Risk: %s" % str(len(self.red_zone_list))  # Count People at Risk
        location = (10, 25)  # Set the location of the displayed text
        cv2.putText(img, text, location, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1,
                    cv2.LINE_AA)  # Display Text
        
        text = "People awithout mask: %s" % str(masks)  # Count People at Risk
        location = (10, 35)  # Set the location of the displayed text
        cv2.putText(img, text, location, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1,
                cv2.LINE_AA)  # Display Text
        return img
    
    

[INFO] loading face detector model...
[INFO] loading person detector model...


In [166]:
detector.detected_faces

array([[[[0.        , 1.        , 0.9907478 , ..., 0.50769085,
          0.65868026, 0.9929425 ],
         [0.        , 1.        , 0.12922436, ..., 3.9994678 ,
          4.832558  , 4.9834623 ],
         [0.        , 1.        , 0.1231216 , ..., 0.12280336,
          0.98614687, 0.872064  ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [167]:
import cv2
import cvzone
import numpy as np
import io
import traceback

cap = cv2.VideoCapture(0)
cv2.namedWindow("Image", cv2.WINDOW_NORMAL)

detector = Detector()

try:
    while True:
        success, frame = cap.read()
        
        distances = detector.detect_distance_from_camera(frame)
        detector.detect_sd(frame, distances)
        detector.detect_face_mask(frame)
        
        detected_frame = detector.draw_detections(frame)
        cv2.resizeWindow('Image', 600,600)
        cv2.imshow("Image", detected_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            cap.release()
            break
except Exception as errors:
    errors = io.StringIO()
    traceback.print_exc(file=errors)
    contents = str(errors.getvalue())
    print(contents)
    errors.close()
finally:
    cv2.destroyAllWindows()
    cap.release()

Eye distances:  {0: []}
Persons recogized:  {0: [232, 200, 67, 102, 398, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [232, 199, 67, 101, 398, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [231, 199, 66, 100, 397, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [231, 200, 65, 102, 397, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [231, 200, 65, 103, 397, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [231, 201, 67, 104, 396, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [231, 201, 64, 105, 398, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [231, 201, 65, 105, 397, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [230, 201, 62, 105, 398, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [229, 202, 66, 106, 392, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [230, 208, 72, 117, 389, 300]}
Eye distances:  {0: []}
Persons recogized:  {0: [230, 207, 72, 116, 389, 299]}
Eye distances:  {0: []}
Persons recogized:  {0: [234

Eye distances:  {0: []}
Persons recogized:  {0: [225, 203, 127, 108, 323, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [226, 202, 126, 106, 326, 299]}
Eye distances:  {0: []}
Persons recogized:  {0: [225, 201, 124, 104, 327, 299]}
Eye distances:  {0: []}
Persons recogized:  {0: [222, 201, 119, 103, 326, 299]}
Eye distances:  {0: []}
Persons recogized:  {0: [222, 201, 118, 103, 326, 299]}
Eye distances:  {0: []}
Persons recogized:  {0: [222, 201, 119, 104, 326, 299]}
Eye distances:  {0: []}
Persons recogized:  {0: [223, 201, 122, 104, 324, 299]}
Eye distances:  {0: []}
Persons recogized:  {0: [222, 201, 122, 103, 323, 299]}
Eye distances:  {0: []}
Persons recogized:  {0: [221, 201, 121, 103, 321, 299]}
Eye distances:  {0: []}
Persons recogized:  {0: [221, 201, 122, 104, 320, 299]}
Eye distances:  {0: []}
Persons recogized:  {0: [223, 201, 122, 104, 325, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [222, 201, 121, 104, 323, 299]}
Eye distances:  {0: []}
Persons recogize

Eye distances:  {0: []}
Persons recogized:  {0: [116, 183, -1, 65, 233, 301]}
Eye distances:  {0: []}
Persons recogized:  {0: [116, 182, -1, 64, 234, 301]}
Eye distances:  {0: [], 1: []}
Persons recogized:  {0: [116, 182, -1, 63, 234, 301], 1: [335, 261, 291, 224, 380, 299]}
Eye distances:  {0: []}
Persons recogized:  {0: [116, 183, -1, 64, 233, 302]}
Eye distances:  {0: []}
Persons recogized:  {0: [116, 183, -1, 65, 234, 301]}
Eye distances:  {0: []}
Persons recogized:  {0: [117, 183, 0, 64, 235, 302]}
Eye distances:  {0: [], 1: []}
Persons recogized:  {0: [117, 183, 0, 65, 235, 302], 1: [337, 263, 294, 229, 381, 298]}
Eye distances:  {0: [], 1: []}
Persons recogized:  {0: [115, 180, 0, 60, 230, 301], 1: [332, 261, 288, 224, 376, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [110, 174, 1, 50, 220, 298]}
Eye distances:  {0: []}
Persons recogized:  {0: [103, 171, 0, 43, 207, 299]}
Eye distances:  {0: []}
Persons recogized:  {0: [100, 167, 0, 36, 201, 299]}
Eye distances:  {0: []